In [15]:
import pygame
import random
import math
import numpy as np
from scipy.interpolate import CubicSpline
import time

In [16]:
SAFETY_DISTANCE = 20

In [65]:
class Car:
    def __init__(self, trajectory, object_in_front = None):
        self.pos = trajectory.start
        self.direction = trajectory.get_direction()
        self.object_in_front = object_in_front
        self.speed = 2
        self.max_speed = 4
        self.acceleration = 0.02
        self.deceleration = 0
        self.lookahead = self.speed * 20
        self.color = (255, 255, 255)
        self.distance = self.determine_distance()
        
    def euclidean_distance(self, a, b):
        return np.sqrt(np.sum((a - b)**2))
        
    def determine_distance(self):
        return self.euclidean_distance(self.pos,self.object_in_front.pos)
        
    def step(self):
        new_distance = self.determine_distance()
        
        #when there is a car in front
        if isinstance(self.object_in_front, Car):
            if new_distance < self.lookahead + SAFETY_DISTANCE:
                self.speed = max(self.speed/2, 0)
            else:
                self.speed = min(self.speed+self.acceleration, self.max_speed)
        
        #when there is a stoplight in front
        elif isinstance(self.object_in_front, Stoplight):
            if self.object_in_front.red == True:
                if new_distance < 0.01:
                    self.speed = 0
                elif new_distance < self.lookahead:
                    self.speed = max(self.speed/2, 0)
            else:
                self.speed = min(self.speed+self.acceleration, self.max_speed)
            
        self.distance = new_distance
        self.lookahead = self.speed * 20
        self.pos = self.pos + (self.speed*self.direction)

In [66]:
class Stoplight:
    def __init__(self, position, red = True):
        self.pos = position
        self.red = red
        self.color = (255, 0, 0)
        
    def step(self):
        self.red = not self.red
        self.change_color()
        
    def change_color(self):
        if self.red == True:
            self.color = (255, 0, 0)
        else:
            self.color = (0, 255, 0)

In [67]:
class Trajectory:
    def __init__(self, p1, p2):
        self.start = p1
        self.end = p2
        self.color = (0, 255, 255)
        
    def get_direction(self):
        direction_not_norm = self.end - self.start
        return direction_not_norm/np.sqrt(np.sum((direction_not_norm)**2))

In [68]:
trajectory1 = Trajectory(np.array([400, 400]), np.array([200, 200]))
trajectory1.get_direction()

array([-0.70710678, -0.70710678])

In [74]:
time.sleep(5)
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Traffic Simulation")
img = pygame.image.load("intersection.png").convert()


#one stoplight
stoplight1 = Stoplight(np.array([400, 300]))

#one trajectory
trajectory1 = Trajectory(np.array([400, 600]), np.array([400, 300]))

#let's create only one car
car_queue = []
car1 = Car(trajectory1, object_in_front = stoplight1)
car_queue.append(car1)
car2 = Car(trajectory1, object_in_front = car1)
car_queue.append(car2)

running = True
iteration = 0
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
            
    screen.fill((100, 100, 100))
    pygame.draw.line(screen, trajectory1.color, trajectory1.start, trajectory1.end)
    
    #stoplight
    pygame.draw.line(screen, stoplight1.color, stoplight1.pos-np.array([4, 0]), stoplight1.pos+np.array([4, 0]), width = 4)
    if iteration%500 == 0:
        stoplight1.step()
    
    #car
    for car in car_queue:
        pygame.draw.circle(screen, car.color, car.pos, 5)
        car.step()
        if car.pos[1] <= 0:
            car.pos[1] = 600
        
    pygame.display.update() 
    iteration += 1

KeyboardInterrupt: 

In [73]:
pygame.quit()